In [1]:
import numpy as np
import pyeemd as ped
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [2]:
header_fname = u'/media/hao/文件/Jupyter File/UCR_TS_Archive_2015/'

def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

def transform(x_train):
    a = []
    for i in range(x_train.shape[0]):
        X_train = ped.eemd(x_train[i])
        X_train = np.row_stack((x_train[i],X_train))
        a.append(X_train)
    return a

fname = 'synthetic_control'
x_train, y_train = readucr(header_fname+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(header_fname+fname+'/'+fname+'_TEST')

In [3]:
X_train = transform(x_train)
X_test = transform(x_test)

In [4]:
X_train = np.array(X_train)
X_test = np.array(X_test)
print('X_train.shape: ',X_train.shape)
print('X_test.shape: ',X_test.shape)

# turn into array of numpy, so that I can use in the follow-up work

X_train.shape:  (300, 6, 60)
X_test.shape:  (300, 6, 60)


In [5]:
print(X_train.shape[0],X_train.shape[1], X_train.shape[2])
print(X_test.shape[0],X_test.shape[1],X_test.shape[2])

300 6 60
300 6 60


In [14]:
# 全局变量
batch_size = 128
nb_classes = len(np.unique(y_test))
epochs = 200
# input image dimensions
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

In [7]:
# the data, shuffled and split between train and test sets
#(X_train, y_train), (X_test, y_test) = mnist.load_data()
'''
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0]],
       ...,
       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0], 
        ..., 
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8)
'''
X_train

array([[[ -3.76940000e-01,   1.22490000e+00,   3.43870000e-01, ...,
           9.18350000e-01,  -1.31170000e+00,  -1.19710000e+00],
        [ -4.42260053e-01,   3.69372682e-01,  -3.19439096e-01, ...,
           5.23777444e-01,  -5.25900592e-01,   4.46585949e-01],
        [ -3.82021023e-01,   3.29412584e-01,   2.01652294e-01, ...,
           4.54724682e-01,  -6.41720500e-01,  -1.40217778e+00],
        [  1.04766288e-01,   1.46498223e-01,   7.38122649e-02, ...,
          -8.35486141e-03,  -4.18466952e-02,  -1.39482849e-01],
        [ -1.95607961e-01,  -1.29142270e-01,  -6.51052955e-02, ...,
          -7.27095980e-02,  -1.14592727e-01,  -1.57457788e-01],
        [  5.39125284e-01,   5.06330310e-01,   4.73741415e-01, ...,
           2.34504733e-02,   3.53327104e-02,   4.71159426e-02]],

       [[  6.44410000e-01,   4.13270000e-01,  -8.62280000e-01, ...,
           1.39810000e+00,   8.57420000e-02,   2.49970000e-02],
        [ -1.69332970e-01,   3.76683405e-01,  -2.37503339e-01, ...,
      

In [8]:
X_train.shape  # output (300, 6, 60)
X_test.shape   # output (300, 6, 60)
y_test.shape   # output (300,)

(300,)

In [9]:
# 根据不同的backend定下不同的格式
if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [10]:
#X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
#X_train /= 255
#X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (300, 6, 60, 1)
300 train samples
300 test samples


In [49]:
y_train

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4

In [11]:
# 转换为one_hot类型
y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# here comes a problem that the number are begin from 1 to 6, not 0 to 5
# so I need to add 1 in nb_classes

In [12]:
Y_test.shape

(300, 6)

In [15]:
#构建模型
model = Sequential()
"""
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='same',
                        input_shape=input_shape))
"""
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]),
                        padding='same',
                        input_shape=input_shape)) # 卷积层1
model.add(Activation('relu')) #激活层
model.add(Convolution2D(nb_filters, (kernel_size[0], kernel_size[1]))) #卷积层2
model.add(Activation('relu')) #激活层
model.add(MaxPooling2D(pool_size=pool_size)) #池化层
model.add(Dropout(0.25)) #神经元随机失活
model.add(Flatten()) #拉成一维数据
model.add(Dense(128)) #全连接层1
model.add(Activation('relu')) #激活层
model.add(Dropout(0.5)) #随机失活
model.add(Dense(nb_classes)) #全连接层2
model.add(Activation('softmax')) #Softmax评分

#编译模型
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
#训练模型
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
          verbose=1, validation_data=(X_test, Y_test))
#评估模型
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 300 samples, validate on 300 samples
Epoch 1/200
300/300 [==============================] - 0s - loss: 1.7703 - acc: 0.2233 - val_loss: 1.6558 - val_acc: 0.6767
Epoch 2/200
300/300 [==============================] - 0s - loss: 1.6268 - acc: 0.4300 - val_loss: 1.4905 - val_acc: 0.6800
Epoch 3/200
300/300 [==============================] - 0s - loss: 1.4695 - acc: 0.6067 - val_loss: 1.2877 - val_acc: 0.7467
Epoch 4/200
300/300 [==============================] - 0s - loss: 1.2676 - acc: 0.6300 - val_loss: 1.0808 - val_acc: 0.6833
Epoch 5/200
300/300 [==============================] - 0s - loss: 1.0386 - acc: 0.6967 - val_loss: 0.8483 - val_acc: 0.8167
Epoch 6/200
300/300 [==============================] - 0s - loss: 0.8242 - acc: 0.7700 - val_loss: 0.6777 - val_acc: 0.8000
Epoch 7/200
300/300 [==============================] - 0s - loss: 0.7170 - acc: 0.7800 - val_loss: 0.5313 - val_acc: 0.9133
Epoch 8/200
300/300 [==============================] - 0s - loss: 0.6244 - acc: 0.7967

300/300 [==============================] - 0s - loss: 0.0183 - acc: 0.9967 - val_loss: 0.0220 - val_acc: 0.9933
Epoch 67/200
300/300 [==============================] - 0s - loss: 0.0218 - acc: 0.9933 - val_loss: 0.0211 - val_acc: 0.9900
Epoch 68/200
300/300 [==============================] - 0s - loss: 0.0222 - acc: 0.9967 - val_loss: 0.0215 - val_acc: 0.9867
Epoch 69/200
300/300 [==============================] - 0s - loss: 0.0234 - acc: 0.9967 - val_loss: 0.0206 - val_acc: 0.9867
Epoch 70/200
300/300 [==============================] - 0s - loss: 0.0260 - acc: 0.9900 - val_loss: 0.0207 - val_acc: 0.9933
Epoch 71/200
300/300 [==============================] - 0s - loss: 0.0129 - acc: 1.0000 - val_loss: 0.0203 - val_acc: 0.9933
Epoch 72/200
300/300 [==============================] - 0s - loss: 0.0180 - acc: 0.9967 - val_loss: 0.0220 - val_acc: 0.9900
Epoch 73/200
300/300 [==============================] - 0s - loss: 0.0324 - acc: 0.9867 - val_loss: 0.0202 - val_acc: 0.9900
Epoch 74/200


300/300 [==============================] - 0s - loss: 0.0084 - acc: 1.0000 - val_loss: 0.0125 - val_acc: 0.9933
Epoch 132/200
300/300 [==============================] - 0s - loss: 0.0039 - acc: 1.0000 - val_loss: 0.0134 - val_acc: 0.9933
Epoch 133/200
300/300 [==============================] - 0s - loss: 0.0060 - acc: 1.0000 - val_loss: 0.0127 - val_acc: 0.9933
Epoch 134/200
300/300 [==============================] - 0s - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0124 - val_acc: 0.9933
Epoch 135/200
300/300 [==============================] - 0s - loss: 0.0032 - acc: 1.0000 - val_loss: 0.0111 - val_acc: 0.9933
Epoch 136/200
300/300 [==============================] - 0s - loss: 0.0060 - acc: 1.0000 - val_loss: 0.0127 - val_acc: 0.9933
Epoch 137/200
300/300 [==============================] - 0s - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0121 - val_acc: 0.9933
Epoch 138/200
300/300 [==============================] - 0s - loss: 0.0026 - acc: 1.0000 - val_loss: 0.0117 - val_acc: 0.9967
Epoch 

300/300 [==============================] - 0s - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0078 - val_acc: 1.0000
Epoch 197/200
300/300 [==============================] - 0s - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0077 - val_acc: 1.0000
Epoch 198/200
300/300 [==============================] - 0s - loss: 0.0046 - acc: 0.9967 - val_loss: 0.0100 - val_acc: 0.9933
Epoch 199/200
300/300 [==============================] - 0s - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0080 - val_acc: 0.9933
Epoch 200/200
300/300 [==============================] - 0s - loss: 0.0041 - acc: 1.0000 - val_loss: 0.0092 - val_acc: 0.9933
Test score: 0.00919696692998
Test accuracy: 0.993333333333
